In [3]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import os
from multiprocessing import Pool, cpu_count, Manager, Value

def scarica(nome_simbolo):
    try:
        if os.path.exists(f'tickers/{nome_simbolo}.parquet'):
            ticker = pd.read_parquet(f'tickers/{nome_simbolo}.parquet')
            
        else:
            ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
            ticker.index = ticker.index.date
            ticker = fx.crea_indicatori(ticker)
            ticker.dropna(axis=0, inplace=True)
        return nome_simbolo, ticker
    except Exception as e:
        return nome_simbolo, str(e)

def modifica_target(nome_simbolo):
    try:
        ticker = pd.read_parquet(f'tickers/{nome_simbolo}.parquet')
        ticker.drop("Target", axis=1, inplace=True)
        ticker = fx.imposta_target(ticker)
        return nome_simbolo, ticker
    except Exception as e:
        return nome_simbolo, str(e)

def callback_result(result, totale_processati, tot_tickers):
    nome_simbolo, ticker_or_error = result
    if isinstance(ticker_or_error, pd.DataFrame):
        ticker_or_error.to_parquet(f'tickers/{nome_simbolo}.parquet')
        print(f"{totale_processati.value}/{tot_tickers}) Completato ticker {nome_simbolo}")
    else:
        print(f"Errore per {nome_simbolo}: {ticker_or_error}")

    with totale_processati.get_lock(): 
        totale_processati.value += 1
    with open(f"tickers/_indice.txt", 'w') as f:
        f.write(str(totale_processati.value))


def scarica_tickers(lista_tickers):
    tot_tickers = len(lista_tickers)

    if os.path.exists('tickers/'):
        try:
            with open(f"tickers/_indice.txt", 'r') as f:
                inizio = int(f.read().strip())  
        except FileNotFoundError:
            inizio = 0  # Se il file non esiste, inizia da 0    
        print(f'inizio={inizio}, n_tickers={tot_tickers}')
    else:
        os.makedirs('tickers/')
        inizio = 0

    totale_processati = Value('i', inizio)  
        
    if inizio <= len(lista_tickers):
        with Pool(cpu_count()) as p:
            for i in range(inizio, tot_tickers):
                nome_simbolo = lista_tickers.iloc[i]["Ticker"]
                task = (nome_simbolo,)
                p.apply_async(scarica, args=task, callback=lambda result: callback_result(result, totale_processati, tot_tickers))
                #p.apply_async(modifica_target, args=task, callback=lambda result: callback_result(result, totale_processati, tot_tickers))

            p.close()
            p.join()




In [4]:
lista_tickers = pd.read_parquet("lista_ticker.parquet")
scarica_tickers(lista_tickers)

inizio=0, n_tickers=2581
0/2581) Completato ticker OOMA
1/2581) Completato ticker CHEF
2/2581) Completato ticker ZIP
3/2581) Completato ticker CE
4/2581) Completato ticker CARR
5/2581) Completato ticker YPF
6/2581) Completato ticker PII
7/2581) Completato ticker PPBI
8/2581) Completato ticker IRTC
9/2581) Completato ticker NGVT
10/2581) Completato ticker SCPL
11/2581) Completato ticker DV
12/2581) Completato ticker AVY
13/2581) Completato ticker WU
14/2581) Completato ticker NABL
15/2581) Completato ticker SCS
16/2581) Completato ticker AWR
17/2581) Completato ticker HQY
18/2581) Completato ticker PODD
19/2581) Completato ticker BKE
20/2581) Completato ticker DG
21/2581) Completato ticker TSCO
22/2581) Completato ticker UVE
23/2581) Completato ticker KRG
24/2581) Completato ticker BBVA
25/2581) Completato ticker IBP
26/2581) Completato ticker DRVN
27/2581) Completato ticker CERE
28/2581) Completato ticker UI
29/2581) Completato ticker FTV
30/2581) Completato ticker VRRM
31/2581) Comple